In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import qokit.fur as qk_fur

In [3]:
number_of_qubits = 5
hamiltonian_seed = 42
numpy_rng = np.random.default_rng()
#possibly modify
qubit_pairs = [(i,j) for i in range(number_of_qubits) for j in range(i+1, number_of_qubits)]
random_coefficients = numpy_rng.uniform(-1, 
                                        1, 
                                        size=len(qubit_pairs)).tolist()
#this is qokit's format
random_hamiltonian = [(coeff, pair) for pair, coeff in zip(qubit_pairs, random_coefficients)]
random_hamiltonian

[(-0.6208471398603623, (0, 1)),
 (-0.601169181362821, (0, 2)),
 (-0.13964706347161338, (0, 3)),
 (-0.015864413893217355, (0, 4)),
 (-0.04945743310966799, (1, 2)),
 (-0.45782329069329064, (1, 3)),
 (-0.8694682317053581, (1, 4)),
 (0.604961470768749, (2, 3)),
 (0.024604107332821812, (2, 4)),
 (-0.9445169556931237, (3, 4))]

In [4]:

qokit_backend = 'auto'
simclass = qk_fur.choose_simulator(name=qokit_backend)
simulator_qokit = simclass(number_of_qubits,
                       terms=random_hamiltonian)
#this computes all eigenvalues (and is used to speed up the computation)
spectrum = simulator_qokit.get_cost_diagonal()

In [5]:
spectrum

array([-3.06922813, -0.31417253,  0.92596406,  1.1976311 , -3.02710606,
       -2.67672719,  0.7702564 , -1.36275329, -1.19517645,  1.00129089,
        0.96872257,  0.68180136,  1.2667915 ,  1.05858212,  3.2328608 ,
        0.54126286,  0.54126286,  3.2328608 ,  1.05858212,  1.2667915 ,
        0.68180136,  0.96872257,  1.00129089, -1.19517645, -1.36275329,
        0.7702564 , -2.67672719, -3.02710606,  1.1976311 ,  0.92596406,
       -0.31417253, -3.06922813])

In [6]:
#phase separator angle
gammas_j = np.array([0.2*np.pi, 0.3*np.pi])
#mixer angle
betas_j = np.array([0.1*np.pi, 0.2*np.pi])
#multiply by 2 to get exp(-1j*gamma*H) operator [this is not necessary, just convention]
results_qokit = simulator_qokit.simulate_qaoa(gammas_j * 2, betas_j)
#get ket
statevector_ideal = simulator_qokit.get_statevector(results_qokit).reshape(-1, 1)
#get exp value
energy_expected_value = simulator_qokit.get_expectation(results_qokit)
energy_expected_value

np.float64(0.8884394824297738)